In [17]:
import pandas as pd
import os
from tqdm import tqdm

def read_file(file_path):
    """Helper function to read files with proper column handling"""
    try:
        # Read file content
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        
        # Check for "No Available results." in the entire content
        if "No Available results." in content:
            return None, True  # Indicate this is a "no results" file
        
        # If we get here, process the file normally
        lines = [line.strip() for line in content.split('\n') if line.strip()]
        
        # Skip comment lines starting with #
        data_lines = [line for line in lines if not line.startswith('#')]
        
        if not data_lines:
            return pd.DataFrame(), False
        
        # Process each line and split by tabs
        processed_data = []
        for line in data_lines:
            row = [col.strip() for col in line.split('\t') if col.strip()]
            if row:
                processed_data.append(row)
        
        # Ensure all rows have the same number of columns
        if processed_data:
            max_cols = max(len(row) for row in processed_data)
            padded_data = [row + [''] * (max_cols - len(row)) for row in processed_data]
            
            # Create DataFrame with automatic column names
            df = pd.DataFrame(padded_data[1:], columns=padded_data[0] if len(padded_data) > 0 else [f'Column_{i}' for i in range(max_cols)])
            return df, False
        else:
            return pd.DataFrame(), False
    
    except Exception as e:
        raise Exception(f"Error reading file: {str(e)}")

def process_excel_files(input_directory='downloaded_files'):
    # Initialize storage
    no_results_files = []
    error_files = []
    extracted_data = pd.DataFrame(columns=['Column_B', 'Column_D'])
    # Get list of files
    excel_files = [f for f in os.listdir(input_directory) if f.endswith(('.xlsx', '.xls'))]
    
    # Process each file
    for filename in tqdm(excel_files, desc="Processing files"):
        try:
            file_path = os.path.join(input_directory, filename)
            
            # Read and process the file
            df, is_no_results = read_file(file_path)
            
            # Handle no results files
            if is_no_results:
                no_results_files.append(filename)
                continue
            
            # Skip empty DataFrames
            if df is None or df.empty:
                continue
            
            # Process files with data
            if 'miRNAname' in df.columns and 'geneName' in df.columns and 'geneType' in df.columns:
                # Filter for "lncRNA" rows
                lncrna_rows = df[df['geneType'] == "lncRNA"]
                
                if not lncrna_rows.empty:
                    temp_df = lncrna_rows[['miRNAname', 'geneName']]
                    temp_df.columns = ['Column_B', 'Column_D']
                    extracted_data = pd.concat([extracted_data, temp_df], ignore_index=True)
            
        except Exception as e:
            error_files.append(filename)
            print(f"\nError processing {filename}: {str(e)}")
    
    # Save results
    if no_results_files:
        pd.DataFrame({'Filename': no_results_files}).to_excel('data_not_found.xlsx', index=False)
        print(f"\nSaved {len(no_results_files)} files to data_not_found.xlsx")
    
    if error_files:
        pd.DataFrame({'Filename': error_files}).to_excel('error_files.xlsx', index=False)
        print(f"\nSaved {len(error_files)} problematic files to error_files.xlsx")
    
    if not extracted_data.empty:
        extracted_data.to_excel('extracted_data.xlsx', index=False)
        print(f"\nSaved {len(extracted_data)} rows to extracted_data.xlsx")
    
    return len(no_results_files), len(extracted_data), len(error_files)

# Execute the function
if __name__ == "__main__":
    no_results_count, extracted_rows, error_count = process_excel_files()
    print(f"\nProcessing complete!")
    print(f"Files with no results: {no_results_count}")
    print(f"Files with errors: {error_count}")
    print(f"Total rows extracted: {extracted_rows}")

Processing files: 100%|██████████| 11/11 [00:00<00:00, 987.93it/s]


Saved 7 files to data_not_found.xlsx

Saved 232 rows to extracted_data.xlsx

Processing complete!
Files with no results: 7
Files with errors: 0
Total rows extracted: 232
